In [1]:
import pandas as pd
import numpy as np
import gzip,csv
train = pd.read_csv(r'train.csv')
N = float(len(train))
DaysOfWeek = sorted(np.unique(train['DayOfWeek']))
PdDistricts = sorted(np.unique(train['PdDistrict']))
classes = sorted(np.unique(train['Category']))
Sol =['Id']
Sol += classes
trainS = train[['DayOfWeek','PdDistrict','Category']]


In [2]:
dictDay = {}
dictCat = {}
dictDayWithCat={}


In [3]:
for i in DaysOfWeek:
    for j in classes:
        selected = trainS[ trainS['Category']==j][['DayOfWeek']]
        selectedDay = len(trainS[trainS['DayOfWeek']==i][['DayOfWeek']])
        if len(selected)!=0:
            selectedWithDay = selected [ selected['DayOfWeek']==i]['DayOfWeek']
            probDbyC = len(selectedWithDay)/float(len(selected))
            probD = selectedDay/N
            probC = len(selected)/N
            dictDay[i]=probD
            dictCat[j]=probC
            dictDayWithCat[(i,j)]=probDbyC


In [4]:
dictDis = {}
dictDisWithCat={}


In [5]:
for i in PdDistricts:
    for j in classes:
        selected = trainS[ trainS['Category']==j][['PdDistrict']]
        selectedDis = len(trainS[trainS['PdDistrict']==i][['PdDistrict']])
        if len(selected)!=0:
            selectedWithDis = selected [ selected['PdDistrict']==i]['PdDistrict']
            probDbyC = len(selectedWithDis)/float(len(selected))
            probD = selectedDis/N
            dictDis[i]=probD
            dictDisWithCat[(i,j)]=probDbyC

In [6]:
#print len(dictDis),len(dictDisWithCat)
test = pd.read_csv(r'test.csv')
outf = gzip.open(r'Output.csv.gz','w')
fo = csv.writer(outf, lineterminator='\n')
fo.writerow(Sol)
for (idd,dist,day) in zip(test['Id'],test['PdDistrict'],test['DayOfWeek']):
    sol = [idd]
    for _class in classes:
        ans1 = dictDisWithCat[(dist,_class)] * dictDayWithCat[(day,_class)] * dictCat[_class]
        ans2 = dictDay[day] * dictDis[dist]
        ans = ans1/ans2
        sol.append(ans)
    fo.writerow(sol)


In [7]:
#EDA

In [8]:
import zipfile
import matplotlib.pyplot as pl


In [9]:
z = zipfile.ZipFile('train.csv.zip')
print(z.namelist())


['train.csv']


In [10]:
train = pd.read_csv(z.open('train.csv'), parse_dates=['Dates'])

In [11]:
train['Year'] = train['Dates'].map(lambda x: x.year)
train['Week'] = train['Dates'].map(lambda x: x.week)
train['Hour'] = train['Dates'].map(lambda x: x.hour)

In [12]:
print(train.head())

                Dates        Category                      Descript  \
0 2015-05-13 23:53:00        WARRANTS                WARRANT ARREST   
1 2015-05-13 23:53:00  OTHER OFFENSES      TRAFFIC VIOLATION ARREST   
2 2015-05-13 23:33:00  OTHER OFFENSES      TRAFFIC VIOLATION ARREST   
3 2015-05-13 23:30:00   LARCENY/THEFT  GRAND THEFT FROM LOCKED AUTO   
4 2015-05-13 23:30:00   LARCENY/THEFT  GRAND THEFT FROM LOCKED AUTO   

   DayOfWeek PdDistrict      Resolution                    Address  \
0  Wednesday   NORTHERN  ARREST, BOOKED         OAK ST / LAGUNA ST   
1  Wednesday   NORTHERN  ARREST, BOOKED         OAK ST / LAGUNA ST   
2  Wednesday   NORTHERN  ARREST, BOOKED  VANNESS AV / GREENWICH ST   
3  Wednesday   NORTHERN            NONE   1500 Block of LOMBARD ST   
4  Wednesday       PARK            NONE  100 Block of BRODERICK ST   

            X          Y  Year  Week  Hour  
0 -122.425892  37.774599  2015    20    23  
1 -122.425892  37.774599  2015    20    23  
2 -122.424363  37

In [13]:
train.PdDistrict.value_counts().plot(kind='bar', figsize=(8,10))
pl.savefig('district_counts.png')


In [14]:
train['event']=1
weekly_events = train[['Week','Year','event']].groupby(['Year','Week']).count().reset_index()
weekly_events_years = weekly_events.pivot(index='Week', columns='Year', values='event').fillna(method='ffill')
#%matplotlib inline
ax = weekly_events_years.interpolate().plot(title='number of cases every 2 weeks', figsize=(10,6))
pl.savefig('events_every_two_weeks.png')

In [15]:
hourly_events = train[['Hour','event']].groupby(['Hour']).count().reset_index()
hourly_events.plot(kind='bar', figsize=(6, 6))
pl.savefig('hourly_events.png')

In [16]:
hourly_district_events = train[['PdDistrict','Hour','event']].groupby(['PdDistrict','Hour']).count().reset_index()
hourly_district_events_pivot = hourly_district_events.pivot(index='Hour', columns='PdDistrict', values='event').fillna(method='ffill')
hourly_district_events_pivot.interpolate().plot(title='number of cases hourly by district', figsize=(10,6))
pl.savefig('hourly_events_by_district.png')